In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse_open/master/data/telecom_churn.csv')

In [9]:
data.head()

,Account length,Area code,International plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,408,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,415,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [7]:
data.drop(['State', 'Voice mail plan'], axis=1, inplace=True)

In [8]:
data['International plan'] = data['International plan'].map({'Yes':1,'No':0})

In [10]:
y = data['Churn'].astype('int')

In [11]:
X = data.drop(['Churn'], axis=1)

In [12]:
X.shape

(3333, 17)

In [13]:
y.shape

(3333,)

In [14]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=17)

In [18]:
X_train.shape, X_test.shape

((2333, 17), (1000, 17))

In [41]:
first_tree = DecisionTreeClassifier(random_state=17)

In [21]:
from sklearn.model_selection import cross_val_score

In [33]:
cross_val_score(KNN, X_train, y_train,cv = 5).mean()

0.86712740439845226

In [26]:
import numpy as np

In [28]:
from sklearn.neighbors import KNeighborsClassifier

In [32]:
KNN = KNeighborsClassifier()

In [30]:
KNeighborsClassifier?

In [34]:
from sklearn.model_selection import GridSearchCV

In [49]:
tree_params = {'max_depth':np.arange(1,11), 'max_features':np.arange(0.5 , 1, 0.1)}

In [52]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1) 

In [53]:
%%time
tree_grid.fit(X_train, y_train)

Wall time: 12.1 s


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), 'max_features': array([ 0.5,  0.6,  0.7,  0.8,  0.9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [56]:
tree_grid.best_score_, tree_grid.best_params_

(0.94170595799399914, {'max_depth': 6, 'max_features': 0.89999999999999991})

In [63]:
knn_params = {'n_neighbors':[1,2,3,4] + list(range(5,25,2))}

In [64]:
knn_grid = GridSearchCV(KNN, knn_params, cv=5, n_jobs=-1) 

In [65]:
%%time
knn_grid.fit(X_train, y_train)

Wall time: 15.9 s


GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [66]:
knn_grid.best_params_, knn_grid.best_score_

({'n_neighbors': 9}, 0.87312473210458641)

In [68]:
y_predict_tree = tree_grid.predict(X_test)

In [69]:
from sklearn.metrics import accuracy_score

In [70]:
accuracy_score(y_test, y_predict_tree)

0.94599999999999995

In [71]:
1 - np.mean(y)

0.8550855085508551

In [75]:
X.columns

Index(['Account length', 'Area code', 'International plan',
       'Number vmail messages', 'Total day minutes', 'Total day calls',
       'Total day charge', 'Total eve minutes', 'Total eve calls',
       'Total eve charge', 'Total night minutes', 'Total night calls',
       'Total night charge', 'Total intl minutes', 'Total intl calls',
       'Total intl charge', 'Customer service calls'],
      dtype='object')

In [72]:
from sklearn.tree import export_graphviz

In [76]:
export_graphviz(tree_grid.best_estimator_, out_file='telecom.dot', feature_names=X.columns, filled=True)